In [1]:
import numpy as np
import pandas as pd
import os


from sklearn import preprocessing

In [2]:
os.chdir("/home/srishti/Downloads/av_reco_engine")

In [3]:
problem_csv=pd.read_csv("train/problem_data.csv")
user_csv=pd.read_csv("train/user_data.csv")

train_submissions=pd.read_csv("train/train_submissions.csv")

In [4]:
# also import test
test_submissions=pd.read_csv("test_submissions_NeDLEvX.csv")

In [5]:
user_csv.isnull().sum()
user_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3571 entries, 0 to 3570
Data columns (total 11 columns):
user_id                      3571 non-null object
submission_count             3571 non-null int64
problem_solved               3571 non-null int64
contribution                 3571 non-null int64
country                      2418 non-null object
follower_count               3571 non-null int64
last_online_time_seconds     3571 non-null int64
max_rating                   3571 non-null float64
rating                       3571 non-null float64
rank                         3571 non-null object
registration_time_seconds    3571 non-null int64
dtypes: float64(2), int64(6), object(3)
memory usage: 307.0+ KB


In [6]:
# filling nulls in problem level data
nas_in_problem_text = ['tags','level_type']
points_col= ['points']

# replace 'NaN' with 'Unavailable' in these columns
for col in nas_in_problem_text:
    problem_csv[col].fillna('Unavailable',inplace=True)

for col in points_col:
    problem_csv[col].fillna(0,inplace=True)    
    
#similar exercise with user level data
nas_in_user = ['country']

# replace 'NaN' with 'Unavailable' in these columns
for col in nas_in_user:
    user_csv[col].fillna('Unavailable',inplace=True)

In [7]:
train_submissions['ID']= train_submissions['user_id']+"_"+  train_submissions['problem_id'] 
train_data = pd.merge(train_submissions, problem_csv, on = ['problem_id'], how = 'left')
train_data = pd.merge(train_data, user_csv, on = ['user_id'], how = 'left')

In [8]:
# let us also store the target variable
y_train = train_data.attempts_range.values

In [9]:
#create test data
test_data = pd.merge(test_submissions, problem_csv, on = ['problem_id'], how = 'left')
test_data = pd.merge(test_data, user_csv, on = ['user_id'], how = 'left')

#investigating nulls in test df
# in level type
test_nulls=test_data[test_data['level_type'].isnull()]
problem_csv_nulls=problem_csv[problem_csv['level_type'].isnull()]

print(test_nulls['problem_id'].unique())
print(problem_csv_nulls['problem_id'].unique())
#no nulls

[]
[]


In [10]:
test_data[:3]

,ID,user_id,problem_id,level_type,points,tags,submission_count,problem_solved,contribution,country,follower_count,last_online_time_seconds,max_rating,rating,rank,registration_time_seconds
0,user_856_prob_5822,user_856,prob_5822,L,0.0,Unavailable,274,231,0,India,35,1503233908,393.062,362.959,intermediate,1422163139
1,user_2642_prob_2334,user_2642,prob_2334,A,500.0,"brute force,implementation",124,114,0,India,8,1505566590,307.053,146.502,beginner,1439989756
2,user_2557_prob_2920,user_2557,prob_2920,A,500.0,"brute force,implementation",367,283,0,Bangladesh,76,1505585750,321.674,241.399,beginner,1396166615


In [11]:
train_data[:3]

,user_id,problem_id,attempts_range,ID,level_type,points,tags,submission_count,problem_solved,contribution,country,follower_count,last_online_time_seconds,max_rating,rating,rank,registration_time_seconds
0,user_232,prob_6507,1,user_232_prob_6507,B,1000.0,strings,53,47,0,Bangladesh,1,1503633778,307.913,206.709,beginner,1432110935
1,user_3568,prob_2994,3,user_3568_prob_2994,C,1500.0,"brute force,dp",133,118,0,Unavailable,0,1502200122,324.255,235.378,beginner,1454664414
2,user_1600,prob_5071,1,user_1600_prob_5071,A,500.0,implementation,50,44,0,Unavailable,7,1504719542,343.177,229.358,beginner,1473490873


In [12]:
cols_order_desired= list(test_data)
cols_order_desired= cols_order_desired+['attempts_range']
cols_order_desired

['ID',
 'user_id',
 'problem_id',
 'level_type',
 'points',
 'tags',
 'submission_count',
 'problem_solved',
 'contribution',
 'country',
 'follower_count',
 'last_online_time_seconds',
 'max_rating',
 'rating',
 'rank',
 'registration_time_seconds',
 'attempts_range']

In [13]:
train_data=train_data[cols_order_desired]
train_data[:3]

,ID,user_id,problem_id,level_type,points,tags,submission_count,problem_solved,contribution,country,follower_count,last_online_time_seconds,max_rating,rating,rank,registration_time_seconds,attempts_range
0,user_232_prob_6507,user_232,prob_6507,B,1000.0,strings,53,47,0,Bangladesh,1,1503633778,307.913,206.709,beginner,1432110935,1
1,user_3568_prob_2994,user_3568,prob_2994,C,1500.0,"brute force,dp",133,118,0,Unavailable,0,1502200122,324.255,235.378,beginner,1454664414,3
2,user_1600_prob_5071,user_1600,prob_5071,A,500.0,implementation,50,44,0,Unavailable,7,1504719542,343.177,229.358,beginner,1473490873,1


In [14]:
# combine train and test datas in to one dataframe
total_df = pd.concat([train_data,test_data]).reset_index(drop=True)
#saving ntrain and a subset for use later
train_subset=total_df[total_df['attempts_range'].notnull()]
ntrain= train_subset.shape[0]

/home/srishti/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [15]:
total_df[:5]

,ID,attempts_range,contribution,country,follower_count,last_online_time_seconds,level_type,max_rating,points,problem_id,problem_solved,rank,rating,registration_time_seconds,submission_count,tags,user_id
0,user_232_prob_6507,1.0,0,Bangladesh,1,1503633778,B,307.913,1000.0,prob_6507,47,beginner,206.709,1432110935,53,strings,user_232
1,user_3568_prob_2994,3.0,0,Unavailable,0,1502200122,C,324.255,1500.0,prob_2994,118,beginner,235.378,1454664414,133,"brute force,dp",user_3568
2,user_1600_prob_5071,1.0,0,Unavailable,7,1504719542,A,343.177,500.0,prob_5071,44,beginner,229.358,1473490873,50,implementation,user_1600
3,user_2256_prob_703,1.0,23,Indonesia,40,1505175115,A,436.927,500.0,prob_703,233,intermediate,399.083,1447736644,271,"brute force,implementation",user_2256
4,user_2321_prob_356,1.0,0,India,80,1505421849,D,492.546,2000.0,prob_356,135,advanced,472.190,1437724657,155,"dfs and similar,trees",user_2321


In [16]:
#checking for nulls
total_df.isnull().sum()

ID                               0
attempts_range               66555
contribution                     0
country                          0
follower_count                   0
last_online_time_seconds         0
level_type                       0
max_rating                       0
points                           0
problem_id                       0
problem_solved                   0
rank                             0
rating                           0
registration_time_seconds        0
submission_count                 0
tags                             0
user_id                          0
dtype: int64

In [17]:
#creating features 

#number of problems solved by the user of a particular level of a problem
total_df['level_user_unique_problem_count'] = total_df.groupby(['user_id','level_type'])['problem_id'].transform('nunique')

In [18]:
#number of users belonging to a particular rank attempting the given problem
problem_rank_users_count = total_df.groupby(['problem_id','rank'])['user_id'].count()
problem_rank_users_count=pd.DataFrame(problem_rank_users_count)
problem_rank_users_count.reset_index(inplace=True)
colmns_1=['problem_id', 'rank', 'problem_rank_unique_user_count']
problem_rank_users_count.columns=colmns_1

#creating a pivot table out of the above
users_pivot=pd.pivot_table(problem_rank_users_count,index=["problem_id"], columns=["rank"],values=["problem_rank_unique_user_count"],fill_value=0)
users_pivot.reset_index(inplace=True)
colmns_2=['problem_id', 'users_adv','users_beg','users_exp','users_int']
users_pivot.columns=colmns_2
users_pivot[:5]

#merge it into total_df
total_df = pd.merge(total_df, users_pivot, on = ['problem_id'], how = 'left')

In [19]:
#average attempts range for a user for a particular level
total_df['mean_attempts_per_user_level'] = total_df.groupby(['user_id','level_type'])['attempts_range'].transform('mean')
#filling nulls with 0
total_df['mean_attempts_per_user_level'] = total_df['mean_attempts_per_user_level'].fillna(0)

In [20]:
total_df

,ID,attempts_range,contribution,country,follower_count,last_online_time_seconds,level_type,max_rating,points,problem_id,...,registration_time_seconds,submission_count,tags,user_id,level_user_unique_problem_count,users_adv,users_beg,users_exp,users_int,mean_attempts_per_user_level
0,user_232_prob_6507,1.0,0,Bangladesh,1,1503633778,B,307.913,1000.0,prob_6507,...,1432110935,53,strings,user_232,16,1,13,0,3,1.875000
1,user_3568_prob_2994,3.0,0,Unavailable,0,1502200122,C,324.255,1500.0,prob_2994,...,1454664414,133,"brute force,dp",user_3568,6,30,59,5,87,2.500000
2,user_1600_prob_5071,1.0,0,Unavailable,7,1504719542,A,343.177,500.0,prob_5071,...,1473490873,50,implementation,user_1600,30,270,908,32,757,1.333333
3,user_2256_prob_703,1.0,23,Indonesia,40,1505175115,A,436.927,500.0,prob_703,...,1447736644,271,"brute force,implementation",user_2256,23,154,142,32,237,1.300000
4,user_2321_prob_356,1.0,0,India,80,1505421849,D,492.546,2000.0,prob_356,...,1437724657,155,"dfs and similar,trees",user_2321,11,8,2,0,11,1.500000
5,user_1569_prob_6064,1.0,0,Unavailable,21,1504870431,C,462.729,2000.0,prob_6064,...,1480812045,173,"dp,string suffix structures,trees",user_1569,17,1,0,0,0,1.666667
6,user_3293_prob_1237,1.0,0,Egypt,87,1505561409,B,398.222,1000.0,prob_1237,...,1439756111,855,implementation,user_3293,24,1,10,0,10,1.928571
7,user_915_prob_4125,2.0,0,Brazil,21,1505581625,B,433.486,1000.0,prob_4125,...,1464182831,254,"greedy,sortings",user_915,24,30,146,1,239,1.500000
8,user_2032_prob_1943,1.0,0,India,28,1505534677,C,387.328,0.0,prob_1943,...,1443374825,216,greedy,user_2032,33,3,12,0,14,1.450000
9,user_1410_prob_3935,1.0,0,Bangladesh,56,1505556770,C,315.367,1000.0,prob_3935,...,1384505273,390,"greedy,sortings",user_1410,16,38,197,0,249,2.800000


In [21]:
train_data[:3]

,ID,user_id,problem_id,level_type,points,tags,submission_count,problem_solved,contribution,country,follower_count,last_online_time_seconds,max_rating,rating,rank,registration_time_seconds,attempts_range
0,user_232_prob_6507,user_232,prob_6507,B,1000.0,strings,53,47,0,Bangladesh,1,1503633778,307.913,206.709,beginner,1432110935,1
1,user_3568_prob_2994,user_3568,prob_2994,C,1500.0,"brute force,dp",133,118,0,Unavailable,0,1502200122,324.255,235.378,beginner,1454664414,3
2,user_1600_prob_5071,user_1600,prob_5071,A,500.0,implementation,50,44,0,Unavailable,7,1504719542,343.177,229.358,beginner,1473490873,1


In [22]:
test_data[:3]

,ID,user_id,problem_id,level_type,points,tags,submission_count,problem_solved,contribution,country,follower_count,last_online_time_seconds,max_rating,rating,rank,registration_time_seconds
0,user_856_prob_5822,user_856,prob_5822,L,0.0,Unavailable,274,231,0,India,35,1503233908,393.062,362.959,intermediate,1422163139
1,user_2642_prob_2334,user_2642,prob_2334,A,500.0,"brute force,implementation",124,114,0,India,8,1505566590,307.053,146.502,beginner,1439989756
2,user_2557_prob_2920,user_2557,prob_2920,A,500.0,"brute force,implementation",367,283,0,Bangladesh,76,1505585750,321.674,241.399,beginner,1396166615


In [23]:
# normalizing submission and problem count by applying log
total_df['submission_log'] = np.log1p(total_df.submission_count)
total_df['problems_log'] = np.log1p(total_df.problem_solved)

# squaring these
total_df['submission_square'] = total_df['submission_count'] ** 2
total_df['problems_square'] = total_df['problem_solved'] ** 2
# doing it for contri too
total_df['contri_square'] = total_df['contribution'] ** 2

# root of these
total_df['submission_root'] = total_df['submission_count'] ** 0.5
total_df['problems_root'] = total_df['problem_solved'] ** 0.5

# average submission per problem
total_df['avg_submissions_per_problem'] = total_df['submission_count']//total_df['problem_solved']

In [24]:
test=total_df['tags'].unique()
print(test.shape[0])
arr_strings=[]


for i in range(0,(test.shape[0]-1)) :
    test_part=str(test[i]).split(",")
    for j in range(0,len(test_part)):
        arr_strings.append(test_part[j])
        
final_list_of_terms= list(set(arr_strings))
terms_df=pd.DataFrame(final_list_of_terms)
terms_df.columns= ['terms']

def replace_asterisk(val):
    if isinstance(val, str):
        return(val.replace('*', ''))
    else:
        return(val)    

def replace_hyphen(val):
    if isinstance(val, str):
        return(val.replace('-', '_'))
    else:
        return(val)    

def remove_extraspace(val):
        if isinstance(val, str):
            return(val.strip())
        else:
            return(val) 

def replace_space(val):
        if isinstance(val, str):
            return(val.replace(' ', '_'))
        else:
            return(val)
        
def add_flag(val):
        prefix= "flag_"
        return prefix+val       

terms_df['terms'] = terms_df['terms'].apply(replace_asterisk)
terms_df['terms'] = terms_df['terms'].apply(replace_hyphen)
terms_df['terms'] = terms_df['terms'].apply(remove_extraspace)
terms_df['terms'] = terms_df['terms'].apply(replace_space)
terms_df['terms'] = terms_df['terms'].apply(add_flag)

terms_amended=terms_df['terms'].tolist()        

876


In [25]:
print(len(terms_amended))
terms_amended

37


['flag_graphs',
 'flag_fft',
 'flag_math',
 'flag_shortest_paths',
 'flag_graph_matchings',
 'flag_dfs_and_similar',
 'flag_special',
 'flag_number_theory',
 'flag_strings',
 'flag_string_suffix_structures',
 'flag_dsu',
 'flag_meet_in_the_middle',
 'flag_hashing',
 'flag_bitmasks',
 'flag_geometry',
 'flag_brute_force',
 'flag_data_structures',
 'flag_trees',
 'flag_games',
 'flag_binary_search',
 'flag_combinatorics',
 'flag_two_pointers',
 'flag_implementation',
 'flag_constructive_algorithms',
 'flag_2_sat',
 'flag_dp',
 'flag_matrices',
 'flag_schedules',
 'flag_probabilities',
 'flag_flows',
 'flag_greedy',
 'flag_ternary_search',
 'flag_expression_parsing',
 'flag_divide_and_conquer',
 'flag_Unavailable',
 'flag_sortings',
 'flag_chinese_remainder_theorem']

In [26]:
total_df_copy=total_df.copy()

In [27]:
#create flag columns for presence of tags
for idx,val in enumerate(terms_amended):
    #get the term being checked
    str_check=final_list_of_terms[idx]
    #associated column name
    col_name= terms_amended[idx]
    #populate the column: if present 1, else 0
    total_df_copy[col_name]=total_df_copy['tags'].apply(lambda x: 0 if str(x).find(str_check, 0)==-1 else 1)
    
total_df_copy[:10]   

,ID,attempts_range,contribution,country,follower_count,last_online_time_seconds,level_type,max_rating,points,problem_id,...,flag_schedules,flag_probabilities,flag_flows,flag_greedy,flag_ternary_search,flag_expression_parsing,flag_divide_and_conquer,flag_Unavailable,flag_sortings,flag_chinese_remainder_theorem
0,user_232_prob_6507,1.0,0,Bangladesh,1,1503633778,B,307.913,1000.0,prob_6507,...,0,0,0,0,0,0,0,0,0,0
1,user_3568_prob_2994,3.0,0,Unavailable,0,1502200122,C,324.255,1500.0,prob_2994,...,0,0,0,0,0,0,0,0,0,0
2,user_1600_prob_5071,1.0,0,Unavailable,7,1504719542,A,343.177,500.0,prob_5071,...,0,0,0,0,0,0,0,0,0,0
3,user_2256_prob_703,1.0,23,Indonesia,40,1505175115,A,436.927,500.0,prob_703,...,0,0,0,0,0,0,0,0,0,0
4,user_2321_prob_356,1.0,0,India,80,1505421849,D,492.546,2000.0,prob_356,...,0,0,0,0,0,0,0,0,0,0
5,user_1569_prob_6064,1.0,0,Unavailable,21,1504870431,C,462.729,2000.0,prob_6064,...,0,0,0,0,0,0,0,0,0,0
6,user_3293_prob_1237,1.0,0,Egypt,87,1505561409,B,398.222,1000.0,prob_1237,...,0,0,0,0,0,0,0,0,0,0
7,user_915_prob_4125,2.0,0,Brazil,21,1505581625,B,433.486,1000.0,prob_4125,...,0,0,0,1,0,0,0,0,1,0
8,user_2032_prob_1943,1.0,0,India,28,1505534677,C,387.328,0.0,prob_1943,...,0,0,0,1,0,0,0,0,0,0
9,user_1410_prob_3935,1.0,0,Bangladesh,56,1505556770,C,315.367,1000.0,prob_3935,...,0,0,0,1,0,0,0,0,1,0


In [28]:
total_df_copy.info()
total_df_copy.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 221850 entries, 0 to 221849
Data columns (total 68 columns):
ID                                 221850 non-null object
attempts_range                     155295 non-null float64
contribution                       221850 non-null int64
country                            221850 non-null object
follower_count                     221850 non-null int64
last_online_time_seconds           221850 non-null int64
level_type                         221850 non-null object
max_rating                         221850 non-null float64
points                             221850 non-null float64
problem_id                         221850 non-null object
problem_solved                     221850 non-null int64
rank                               221850 non-null object
rating                             221850 non-null float64
registration_time_seconds          221850 non-null int64
submission_count                   221850 non-null int64
tags                 

ID                                     0
attempts_range                     66555
contribution                           0
country                                0
follower_count                         0
last_online_time_seconds               0
level_type                             0
max_rating                             0
points                                 0
problem_id                             0
problem_solved                         0
rank                                   0
rating                                 0
registration_time_seconds              0
submission_count                       0
tags                                   0
user_id                                0
level_user_unique_problem_count        0
users_adv                              0
users_beg                              0
users_exp                              0
users_int                              0
mean_attempts_per_user_level           0
submission_log                         0
problems_log    

In [29]:
total_df_copy[:3]

,ID,attempts_range,contribution,country,follower_count,last_online_time_seconds,level_type,max_rating,points,problem_id,...,flag_schedules,flag_probabilities,flag_flows,flag_greedy,flag_ternary_search,flag_expression_parsing,flag_divide_and_conquer,flag_Unavailable,flag_sortings,flag_chinese_remainder_theorem
0,user_232_prob_6507,1.0,0,Bangladesh,1,1503633778,B,307.913,1000.0,prob_6507,...,0,0,0,0,0,0,0,0,0,0
1,user_3568_prob_2994,3.0,0,Unavailable,0,1502200122,C,324.255,1500.0,prob_2994,...,0,0,0,0,0,0,0,0,0,0
2,user_1600_prob_5071,1.0,0,Unavailable,7,1504719542,A,343.177,500.0,prob_5071,...,0,0,0,0,0,0,0,0,0,0


In [30]:
#drop selected columns
cols_to_drop1= ['user_id', 'problem_id']
total_df_copy.drop(cols_to_drop1, axis=1, inplace=True)
train_data.drop(cols_to_drop1, axis=1, inplace=True)

#drop selected columns- round 2
cols_to_drop2= ['tags']
total_df_copy.drop(cols_to_drop2, axis=1, inplace=True)

In [31]:
#create index using 'id'
#this is because the id column will not be used in the algorithm. yet it is needed to identify the unique combination
input_df= total_df_copy.set_index('ID')


# dropping attempts range
input_df.drop(['attempts_range'], axis=1, inplace=True)  

#check for nulls again
input_df.isnull().sum()

contribution                       0
country                            0
follower_count                     0
last_online_time_seconds           0
level_type                         0
max_rating                         0
points                             0
problem_solved                     0
rank                               0
rating                             0
registration_time_seconds          0
submission_count                   0
level_user_unique_problem_count    0
users_adv                          0
users_beg                          0
users_exp                          0
users_int                          0
mean_attempts_per_user_level       0
submission_log                     0
problems_log                       0
submission_square                  0
problems_square                    0
contri_square                      0
submission_root                    0
problems_root                      0
avg_submissions_per_problem        0
flag_graphs                        0
f

In [32]:
input_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 221850 entries, user_232_prob_6507 to user_474_prob_6497
Data columns (total 63 columns):
contribution                       221850 non-null int64
country                            221850 non-null object
follower_count                     221850 non-null int64
last_online_time_seconds           221850 non-null int64
level_type                         221850 non-null object
max_rating                         221850 non-null float64
points                             221850 non-null float64
problem_solved                     221850 non-null int64
rank                               221850 non-null object
rating                             221850 non-null float64
registration_time_seconds          221850 non-null int64
submission_count                   221850 non-null int64
level_user_unique_problem_count    221850 non-null int64
users_adv                          221850 non-null int64
users_beg                          221850 non-null int64
u

In [33]:
# create a list of ordinal variables
ordinal_variables=['level_type','rank']
# label encoder
le = preprocessing.LabelEncoder()

for c in ordinal_variables:
    le.fit(input_df[c])
    input_df[c] = le.transform(input_df[c])
    
    
# create a list of categorical columns for one hot encoding
cat_variables= ['country']

# One-Hot encoding to convert categorical columns to numeric
print('start one-hot encoding')

input_df = pd.get_dummies(input_df, prefix = cat_variables,
                         columns = cat_variables)

print('one-hot encoding done')

  

start one-hot encoding
one-hot encoding done


In [34]:
input_df[:10]

,contribution,follower_count,last_online_time_seconds,level_type,max_rating,points,problem_solved,rank,rating,registration_time_seconds,...,country_Trinidad and Tobago,country_Tunisia,country_Turkmenistan,country_Ukraine,country_Unavailable,country_United Kingdom,country_United States,country_Uzbekistan,country_Venezuela,country_Vietnam
ID,,,,,,,,,,,,,,,,,,,,,
user_232_prob_6507,0,1,1503633778,1,307.913,1000.0,47,1,206.709,1432110935,...,0,0,0,0,0,0,0,0,0,0
user_3568_prob_2994,0,0,1502200122,2,324.255,1500.0,118,1,235.378,1454664414,...,0,0,0,0,1,0,0,0,0,0
user_1600_prob_5071,0,7,1504719542,0,343.177,500.0,44,1,229.358,1473490873,...,0,0,0,0,1,0,0,0,0,0
user_2256_prob_703,23,40,1505175115,0,436.927,500.0,233,3,399.083,1447736644,...,0,0,0,0,0,0,0,0,0,0
user_2321_prob_356,0,80,1505421849,3,492.546,2000.0,135,0,472.190,1437724657,...,0,0,0,0,0,0,0,0,0,0
user_1569_prob_6064,0,21,1504870431,2,462.729,2000.0,161,0,462.729,1480812045,...,0,0,0,0,1,0,0,0,0,0
user_3293_prob_1237,0,87,1505561409,1,398.222,1000.0,811,3,368.979,1439756111,...,0,0,0,0,0,0,0,0,0,0
user_915_prob_4125,0,21,1505581625,1,433.486,1000.0,217,3,395.356,1464182831,...,0,0,0,0,0,0,0,0,0,0
user_2032_prob_1943,0,28,1505534677,2,387.328,0.0,199,3,387.328,1443374825,...,0,0,0,0,0,0,0,0,0,0


In [35]:
# Splitting the train and test datasets
train = input_df[:ntrain]
test = input_df[ntrain:]

In [36]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import f1_score

In [37]:
# training a KNN classifier 
from sklearn.neighbors import KNeighborsClassifier 

In [38]:
knn = KNeighborsClassifier(n_neighbors = 5).fit(train.values, y_train) 
  
# creating a confusion matrix 
knn_predictions = knn.predict(test.values)  
cm_knn = confusion_matrix(y_train, knn.predict(train.values))

f1_score(y_train, knn.predict(train.values), average='weighted')

0.5820150670434688

In [39]:
knn.classes_

array([1, 2, 3, 4, 5, 6])

In [40]:
import xgboost as xgb

In [41]:
xgb_model = xgb.XGBClassifier(max_depth=5, learning_rate=0.01, n_estimators=500, gamma=0, 
                        min_child_weight=1, subsample=0.8, colsample_bytree=0.8, reg_alpha=0.005)

In [42]:
xgb_model=xgb_model.fit(train.values, y_train) 
xgb_predictions = xgb_model.predict(test.values)

/home/srishti/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [43]:
f1_score(y_train, xgb_model.predict(train.values), average='weighted')

/home/srishti/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.513259994883282